In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, TimeSeriesSplit
from xgboost import XGBRegressor
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
import scipy.stats as stats
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import MinMaxScaler

In [2]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


Kaggle credentials set.
Kaggle credentials successfully validated.


In [3]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

walmart_recruiting_store_sales_forecasting_path = kagglehub.competition_download('walmart-recruiting-store-sales-forecasting')

print('Data source import complete.')


100%|██████████| 2.70M/2.70M [00:00<00:00, 147MB/s]

Extracting files...
Data source import complete.


In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
! mkdir ~/.kaggle
!cp /content/drive/MyDrive/ML/kaggle.json ~/.kaggle/kaggle.json
! chmod 600 ~/.kaggle/kaggle.json
! kaggle competitions download -c walmart-recruiting-store-sales-forecasting
! unzip walmart-recruiting-store-sales-forecasting


  0% 0.00/2.70M [00:00<?, ?B/s]
100% 2.70M/2.70M [00:00<00:00, 911MB/s]
Archive:  walmart-recruiting-store-sales-forecasting.zip
  inflating: features.csv.zip        
  inflating: sampleSubmission.csv.zip  
  inflating: stores.csv              
  inflating: test.csv.zip            
  inflating: train.csv.zip           


In [12]:
stores = pd.read_csv('stores.csv')
train = pd.read_csv("train.csv.zip")
features = pd.read_csv('features.csv.zip')
sample = pd.read_csv('sampleSubmission.csv.zip')
test = pd.read_csv('test.csv.zip')

In [13]:
train_ = pd.merge(train, features, on=['Store', 'Date', 'IsHoliday'], how='left')
test_df = pd.merge(test, features, on=['Store', 'Date', 'IsHoliday'], how='left')
train_ = pd.merge(train_, stores, on='Store', how='left')
test_df = pd.merge(test_df, stores, on='Store', how='left')


In [11]:
import gc

print("\n--- Merged Train Data Info ---")
print(train_.info())
print("\n--- Merged Test Data Info ---")
print(test_df.info())

del train, test, features, stores
gc.collect()


--- Merged Train Data Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 421570 entries, 0 to 421569
Data columns (total 16 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Store         421570 non-null  int64  
 1   Dept          421570 non-null  int64  
 2   Date          421570 non-null  object 
 3   Weekly_Sales  421570 non-null  float64
 4   IsHoliday     421570 non-null  bool   
 5   Temperature   421570 non-null  float64
 6   Fuel_Price    421570 non-null  float64
 7   MarkDown1     150681 non-null  float64
 8   MarkDown2     111248 non-null  float64
 9   MarkDown3     137091 non-null  float64
 10  MarkDown4     134967 non-null  float64
 11  MarkDown5     151432 non-null  float64
 12  CPI           421570 non-null  float64
 13  Unemployment  421570 non-null  float64
 14  Type          421570 non-null  object 
 15  Size          421570 non-null  int64  
dtypes: bool(1), float64(10), int64(3), object(2)
memory usage: 4

0

In [ ]:
class MissingMarkdownHandler:
    """
    Handles missing markdown values by replacing NaNs with 0s and creating
    boolean flags to distinguish between actual 0s and missing information.
    """
    def __init__(self):
        self.markdown_cols = None

    def fit(self, X, y=None):
        # Identify markdown columns
        self.markdown_cols = [col for col in X.columns if 'markdown' in col.lower()]
        return self

    def transform(self, X):
        X_copy = X.copy()

        for col in self.markdown_cols:
            if col in X_copy.columns:
                # Create boolean flag for missing information
                X_copy[f'{col}_was_missing'] = X_copy[col].isna().astype(int)

                # Replace NaNs with 0s (better than median for independence assumption)
                X_copy[col] = X_copy[col].fillna(0)

        return X_copy

    def fit_transform(self, X, y=None):
        return self.fit(X).transform(X)



In [ ]:
class MissingValueImputer:
    """
    Handles missing values in non-markdown features using forward-fill and backward-fill
    strategy, which is more relevant for time-series data than mean/median.
    """
    def __init__(self):
        self.non_markdown_cols = None

    def fit(self, X, y=None):
        # Identify non-markdown columns that might have missing values
        self.non_markdown_cols = [col for col in X.columns
                                 if 'markdown' not in col.lower() and X[col].dtype in ['float64', 'int64']]
        return self

    def transform(self, X):
        X_copy = X.copy()

        # Sort by date for proper forward/backward fill
        if 'Date' in X_copy.columns:
            X_copy = X_copy.sort_values('Date')

        for col in self.non_markdown_cols:
            if col in X_copy.columns and X_copy[col].isna().any():
                # Forward fill first (use previous week's information)
                X_copy[col] = X_copy[col].fillna(method='ffill')
                # Backward fill for any remaining NaNs
                X_copy[col] = X_copy[col].fillna(method='bfill')

        return X_copy

    def fit_transform(self, X, y=None):
        return self.fit(X).transform(X)



In [ ]:
class DateFeatureExtractor:
    """
    Extracts comprehensive date features including seasonal patterns,
    cyclic encoding, and holiday proximity features.
    """
    def __init__(self):
        self.holidays = {
            'Christmas': '12-25',
            'Thanksgiving': '11-22',
            'New_Year': '01-01',
            'Independence_Day': '07-04',
            'Labor_Day': '09-01',
        }

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_copy = X.copy()

        # Ensure Date column is datetime
        if 'Date' in X_copy.columns:
            X_copy['Date'] = pd.to_datetime(X_copy['Date'])

            # Basic date features
            X_copy['Year'] = X_copy['Date'].dt.year
            X_copy['Month'] = X_copy['Date'].dt.month
            X_copy['Week'] = X_copy['Date'].dt.isocalendar().week
            X_copy['Day'] = X_copy['Date'].dt.day
            X_copy['DayOfWeek'] = X_copy['Date'].dt.dayofweek

            # Cyclic encoding for seasonal patterns
            X_copy['Month_sin'] = np.sin(2 * np.pi * X_copy['Month'] / 12)
            X_copy['Month_cos'] = np.cos(2 * np.pi * X_copy['Month'] / 12)
            X_copy['Week_sin'] = np.sin(2 * np.pi * X_copy['Week'] / 52)
            X_copy['Week_cos'] = np.cos(2 * np.pi * X_copy['Week'] / 52)

            # Holiday proximity features
            for holiday, date_str in self.holidays.items():
                month, day = map(int, date_str.split('-'))
                holiday_date = pd.to_datetime(f"{X_copy['Year'].iloc[0]}-{month:02d}-{day:02d}")

                days_until = (holiday_date - X_copy['Date']).dt.days
                X_copy[f'days_until_{holiday.lower()}'] = days_until
                X_copy[f'{holiday.lower()}_week'] = (np.abs(days_until) <= 7).astype(int)

        return X_copy

    def fit_transform(self, X, y=None):
        return self.fit(X).transform(X)



In [ ]:
class AdvancedDateFeatureExtractor(BaseEstimator, TransformerMixin):
    """
    Enhanced date feature extractor with holiday detection and seasonal patterns
    """
    def __init__(self, date_column: str = 'Date', include_holidays: bool = True,
                 include_seasonal: bool = True, include_lags: bool = False):
        self.date_column = date_column
        self.include_holidays = include_holidays
        self.include_seasonal = include_seasonal
        self.include_lags = include_lags

    def _is_holiday_period(self, date):
        """Detect major US retail holiday periods"""
        month, day = date.month, date.day

        # Black Friday week (last week of November)
        if month == 11 and day >= 22:
            return 1
        # Christmas season (December)
        elif month == 12:
            return 1
        # New Year
        elif month == 1 and day <= 7:
            return 1
        # Labor Day (first Monday in September) - approximate
        elif month == 9 and day <= 7:
            return 1
        # Memorial Day (last Monday in May) - approximate
        elif month == 5 and day >= 25:
            return 1
        else:
            return 0

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_copy = X.copy()

        if self.date_column not in X_copy.columns:
            raise ValueError(f"Date column '{self.date_column}' not found in DataFrame.")

        # Ensure datetime format
        X_copy[self.date_column] = pd.to_datetime(X_copy[self.date_column])

        # Basic temporal features
        X_copy['Year'] = X_copy[self.date_column].dt.year
        X_copy['Month'] = X_copy[self.date_column].dt.month
        X_copy['Day'] = X_copy[self.date_column].dt.day
        X_copy['DayOfWeek'] = X_copy[self.date_column].dt.dayofweek
        X_copy['Week'] = X_copy[self.date_column].dt.isocalendar().week.astype(int)
        X_copy['Quarter'] = X_copy[self.date_column].dt.quarter
        X_copy['DayOfYear'] = X_copy[self.date_column].dt.dayofyear

        # Cyclical encoding
        X_copy['Month_sin'] = np.sin(2 * np.pi * X_copy['Month'] / 12)
        X_copy['Month_cos'] = np.cos(2 * np.pi * X_copy['Month'] / 12)
        X_copy['Week_sin'] = np.sin(2 * np.pi * X_copy['Week'] / 52)
        X_copy['Week_cos'] = np.cos(2 * np.pi * X_copy['Week'] / 52)
        X_copy['DayOfWeek_sin'] = np.sin(2 * np.pi * X_copy['DayOfWeek'] / 7)
        X_copy['DayOfWeek_cos'] = np.cos(2 * np.pi * X_copy['DayOfWeek'] / 7)

        # Seasonal features
        if self.include_seasonal:
            X_copy['Season'] = X_copy['Month'].map({12: 0, 1: 0, 2: 0,  # Winter
                                                   3: 1, 4: 1, 5: 1,    # Spring
                                                   6: 2, 7: 2, 8: 2,    # Summer
                                                   9: 3, 10: 3, 11: 3}) # Fall

            # Weekend indicator
            X_copy['IsWeekend'] = (X_copy['DayOfWeek'] >= 5).astype(int)

            # Month-end indicator
            X_copy['IsMonthEnd'] = (X_copy[self.date_column].dt.is_month_end).astype(int)
            X_copy['IsMonthStart'] = (X_copy[self.date_column].dt.is_month_start).astype(int)

        # Holiday features
        if self.include_holidays:
            X_copy['IsHolidayPeriod'] = X_copy[self.date_column].apply(self._is_holiday_period)

            # Convert existing IsHoliday to int if it exists
            if 'IsHoliday' in X_copy.columns:
                if X_copy['IsHoliday'].dtype == bool:
                    X_copy['IsHoliday'] = X_copy['IsHoliday'].astype(int)

        # Enhanced markdown features if markdown columns exist
        markdown_cols = [col for col in X_copy.columns if col.startswith('MarkDown') and not col.endswith('_was_missing')]
        if markdown_cols:
            X_copy['Total_MarkDown'] = X_copy[markdown_cols].sum(axis=1)
            X_copy['MarkDown_Intensity'] = X_copy['Total_MarkDown'] / (X_copy['Total_MarkDown'].mean() + 1e-8)
            X_copy['HasMarkDown'] = (X_copy['Total_MarkDown'] > 0).astype(int)

        # Economic features if available
        econ_cols = ['Fuel_Price', 'CPI', 'Unemployment']
        available_econ = [col for col in econ_cols if col in X_copy.columns]

        if len(available_econ) >= 2:
            if 'Fuel_Price' in X_copy.columns and 'CPI' in X_copy.columns:
                X_copy['Fuel_CPI_Ratio'] = X_copy['Fuel_Price'] / (X_copy['CPI'] + 1e-8)

            if 'CPI' in X_copy.columns and 'Unemployment' in X_copy.columns:
                X_copy['Economic_Index'] = (X_copy['CPI'] * 0.4 + (100 - X_copy['Unemployment']) * 0.6) / 100

        return X_copy


In [ ]:
class MarkdownFeatureEngineer:
    """
    Creates markdown-related features including total markdown and intensity.
    """
    def __init__(self):
        self.markdown_cols = None

    def fit(self, X, y=None):
        self.markdown_cols = [col for col in X.columns if 'markdown' in col.lower()
                             and col not in [c for c in X.columns if 'was_missing' in c]]
        return self

    def transform(self, X):
        X_copy = X.copy()

        if self.markdown_cols:
            # Total markdown amount
            X_copy['Total_MarkDown'] = X_copy[self.markdown_cols].sum(axis=1)

            # Markdown intensity (normalized by store size if available)
            if 'Size' in X_copy.columns:
                X_copy['MarkDown_Intensity'] = X_copy['Total_MarkDown'] / (X_copy['Size'] + 1)
            else:
                # Normalize by mean markdown if Size not available
                mean_markdown = X_copy['Total_MarkDown'].mean()
                X_copy['MarkDown_Intensity'] = X_copy['Total_MarkDown'] / (mean_markdown + 1)

        return X_copy

    def fit_transform(self, X, y=None):
        return self.fit(X).transform(X)



In [ ]:
class EconomicFeatureEngineer:
    """
    Creates economic index by combining CPI and Unemployment into unified score.
    """
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_copy = X.copy()

        if 'CPI' in X_copy.columns and 'Unemployment' in X_copy.columns:
            # Normalize both features to 0-1 scale for combination
            cpi_norm = (X_copy['CPI'] - X_copy['CPI'].min()) / (X_copy['CPI'].max() - X_copy['CPI'].min())
            unemployment_norm = (X_copy['Unemployment'] - X_copy['Unemployment'].min()) / (X_copy['Unemployment'].max() - X_copy['Unemployment'].min())

            # Economic index (higher CPI and lower unemployment = better economic conditions)
            X_copy['Economic_Index'] = cpi_norm - unemployment_norm

        return X_copy

    def fit_transform(self, X, y=None):
        return self.fit(X).transform(X)



In [ ]:
class LagFeatureCreator:
    """
    Creates lag features for each (Store, Dept) combination.
    """
    def __init__(self, target_col='Weekly_Sales', lags=[1, 4, 52]):
        self.target_col = target_col
        self.lags = lags

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_copy = X.copy()

        if self.target_col in X_copy.columns and 'Store' in X_copy.columns and 'Dept' in X_copy.columns:
            # Sort by Store, Dept, and Date for proper lag calculation
            if 'Date' in X_copy.columns:
                X_copy = X_copy.sort_values(['Store', 'Dept', 'Date'])

            for lag in self.lags:
                lag_col_name = f'{self.target_col}_lag_{lag}'
                X_copy[lag_col_name] = X_copy.groupby(['Store', 'Dept'])[self.target_col].shift(lag)

        return X_copy

    def fit_transform(self, X, y=None):
        return self.fit(X).transform(X)



In [ ]:
from sklearn.preprocessing import LabelEncoder
class XGBoostLabelEncoder:
    """
    Encodes categorical features using LabelEncoder for XGBoost compatibility.
    """
    def __init__(self):
        self.encoders = {}
        self.categorical_cols = None

    def fit(self, X, y=None):
        # Identify categorical columns
        self.categorical_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()

        for col in self.categorical_cols:
            if col != 'Date':  # Don't encode Date column
                self.encoders[col] = LabelEncoder()
                self.encoders[col].fit(X[col].astype(str))

        return self

    def transform(self, X):
        X_copy = X.copy()

        for col in self.categorical_cols:
            if col in self.encoders and col in X_copy.columns and col != 'Date':
                X_copy[col] = self.encoders[col].transform(X_copy[col].astype(str))

        return X_copy

    def fit_transform(self, X, y=None):
        return self.fit(X).transform(X)

In [ ]:
from typing import Optional, List
from sklearn.base import BaseEstimator, TransformerMixin
import pandas as pd
import numpy as np


class SmartLabelEncoder(BaseEstimator, TransformerMixin):
    """
    Enhanced label encoder with better handling of unseen categories and validation sets
    """
    def __init__(self, categorical_cols: Optional[List[str]] = None,
                 handle_unknown: str = 'ignore', unknown_value: int = -1):
        """
        Parameters:
        - handle_unknown: 'ignore', 'error', or 'use_encoded_value'
        - unknown_value: value to use for unknown categories
        """
        self.categorical_cols = categorical_cols or ['Store', 'Dept', 'Type']
        self.handle_unknown = handle_unknown
        self.unknown_value = unknown_value
        self.label_encoders = {}
        self.category_mappings = {}

    def fit(self, X, y=None):
        for col in self.categorical_cols:
            if col in X.columns:
                # Create robust encoder
                unique_vals = X[col].astype(str).unique()
                unique_vals = unique_vals[pd.notna(unique_vals)]  # Remove NaN values

                # Sort for consistent encoding
                unique_vals = sorted(unique_vals)

                # Create mapping
                self.category_mappings[col] = {val: idx for idx, val in enumerate(unique_vals)}

                # Add unknown category mapping if needed
                if self.handle_unknown == 'use_encoded_value':
                    max_idx = len(unique_vals)
                    self.category_mappings[col]['__UNKNOWN__'] = max_idx

        return self

    def transform(self, X):
        X_copy = X.copy()

        for col in self.categorical_cols:
            if col in X_copy.columns and col in self.category_mappings:
                # Convert to string and handle NaN
                str_col = X_copy[col].astype(str)
                str_col = str_col.replace('nan', '__UNKNOWN__')

                # Map values
                mapping = self.category_mappings[col]

                if self.handle_unknown == 'ignore':
                    # Map known values, keep unknown as is
                    X_copy[col] = str_col.map(mapping).fillna(self.unknown_value)
                elif self.handle_unknown == 'use_encoded_value':
                    # Map all values, use special encoding for unknown
                    X_copy[col] = str_col.apply(lambda x: mapping.get(x, mapping['__UNKNOWN__']))
                else:  # error
                    unknown_vals = set(str_col.unique()) - set(mapping.keys())
                    if unknown_vals:
                        raise ValueError(f"Unknown categories found in column {col}: {unknown_vals}")
                    X_copy[col] = str_col.map(mapping)

        return X_copy

In [ ]:
def wmae_score(y_true, y_pred, weights):
    return np.sum(weights * np.abs(y_true - y_pred)) / np.sum(weights)



In [ ]:
class WMAEScorer:
    """
    Custom scorer for WMAE metric compatible with sklearn's GridSearchCV.
    """
    def __init__(self, weights_col='IsHoliday'):
        self.weights_col = weights_col

    def __call__(self, estimator, X, y):
        y_pred = estimator.predict(X)

        # Create weights (holiday weeks get 5x weight)
        if isinstance(X, pd.DataFrame) and self.weights_col in X.columns:
            weights = X[self.weights_col].apply(lambda x: 5 if x else 1)
        else:
            weights = np.ones(len(y))  # Equal weights if no holiday info

        return -wmae_score(y, y_pred, weights)  # Negative for sklearn (higher is better)



In [ ]:
class DataPreprocessingPipeline:
    """
    Complete preprocessing pipeline that applies all transformations in sequence.
    """
    def __init__(self):
        self.missing_markdown_handler = MissingMarkdownHandler()
        self.missing_value_imputer = MissingValueImputer()
        self.date_feature_extractor = DateFeatureExtractor()
        self.markdown_feature_engineer = MarkdownFeatureEngineer()
        self.economic_feature_engineer = EconomicFeatureEngineer()
        self.lag_feature_creator = LagFeatureCreator()
        self.label_encoder = XGBoostLabelEncoder()

    def fit(self, X, y=None):
        # Fit all transformers in sequence
        X_temp = self.missing_markdown_handler.fit_transform(X)
        X_temp = self.missing_value_imputer.fit_transform(X_temp)
        X_temp = self.date_feature_extractor.fit_transform(X_temp)
        X_temp = self.markdown_feature_engineer.fit_transform(X_temp)
        X_temp = self.economic_feature_engineer.fit_transform(X_temp)
        X_temp = self.lag_feature_creator.fit_transform(X_temp)
        self.label_encoder.fit(X_temp)

        return self

    def transform(self, X):
        X_temp = self.missing_markdown_handler.transform(X)
        X_temp = self.missing_value_imputer.transform(X_temp)
        X_temp = self.date_feature_extractor.transform(X_temp)
        X_temp = self.markdown_feature_engineer.transform(X_temp)
        X_temp = self.economic_feature_engineer.transform(X_temp)
        X_temp = self.lag_feature_creator.transform(X_temp)
        X_temp = self.label_encoder.transform(X_temp)

        return X_temp

    def fit_transform(self, X, y=None):
        return self.fit(X, y).transform(X)



In [ ]:
def create_train_validation_split(df, split_date='2012-09-01', date_col='Date'):
    """
    Split data into train and validation sets based on date.

    Parameters:
    df: DataFrame with date column
    split_date: Date to split on
    date_col: Name of date column

    Returns:
    train_df, val_df
    """
    df[date_col] = pd.to_datetime(df[date_col])
    split_date = pd.to_datetime(split_date)

    train_df = df[df[date_col] < split_date].copy()
    val_df = df[df[date_col] >= split_date].copy()

    print(f"Train set: {len(train_df)} samples")
    print(f"Validation set: {len(val_df)} samples")
    print(f"Train date range: {train_df[date_col].min()} to {train_df[date_col].max()}")
    print(f"Validation date range: {val_df[date_col].min()} to {val_df[date_col].max()}")

    return train_df, val_df



In [ ]:
class XGBoostTrainer:
    """
    XGBoost model trainer with grid search and WMAE optimization.
    """
    def __init__(self, random_state=42):
        self.random_state = random_state
        self.best_model = None
        self.best_params = None
        self.best_score = None

    def grid_search_basic_params(self, X_train, y_train, weights_train=None):
        """
        Grid search for basic XGBoost parameters: learning_rate and max_depth.
        """
        print("Starting grid search for basic parameters...")

        # Parameter grid
        param_grid = {
            'learning_rate': [0.01, 0.1, 0.2],
            'max_depth': [3, 6, 9],
            'n_estimators': [100, 200, 300],
            'random_state': [self.random_state]
        }

        # Base model
        xgb_model = xgb.XGBRegressor(
            objective='reg:squarederror',
            random_state=self.random_state
        )

        # Time series cross-validation
        tscv = TimeSeriesSplit(n_splits=3)

        # Custom WMAE scorer
        wmae_scorer = WMAEScorer()

        # Grid search
        grid_search = GridSearchCV(
            estimator=xgb_model,
            param_grid=param_grid,
            cv=tscv,
            scoring=wmae_scorer,
            n_jobs=-1,
            verbose=1
        )

        grid_search.fit(X_train, y_train)

        self.best_params = grid_search.best_params_
        self.best_score = -grid_search.best_score_  # Convert back to positive WMAE
        self.best_model = grid_search.best_estimator_

        print(f"Best parameters: {self.best_params}")
        print(f"Best WMAE score: {self.best_score:.4f}")

        return self.best_params, self.best_score

    def train_with_custom_params(self, X_train, y_train, **custom_params):
        """
        Train XGBoost with custom parameters.
        """
        print("Training XGBoost with custom parameters...")

        # Default parameters
        default_params = {
            'objective': 'reg:squarederror',
            'learning_rate': 0.1,
            'max_depth': 6,
            'n_estimators': 200,
            'random_state': self.random_state
        }

        # Update with custom parameters
        default_params.update(custom_params)

        # Train model
        self.best_model = xgb.XGBRegressor(**default_params)
        self.best_model.fit(X_train, y_train)

        print(f"Model trained with parameters: {default_params}")

        return self.best_model

    def evaluate_model(self, X_val, y_val, weights_val=None):
        """
        Evaluate the trained model on validation set.
        """
        if self.best_model is None:
            raise ValueError("No model has been trained yet!")

        # Make predictions
        y_pred = self.best_model.predict(X_val)

        # Calculate WMAE
        if weights_val is None:
            # Create weights based on IsHoliday if available
            if isinstance(X_val, pd.DataFrame) and 'IsHoliday' in X_val.columns:
                weights_val = X_val['IsHoliday'].apply(lambda x: 5 if x else 1)
            else:
                weights_val = np.ones(len(y_val))

        wmae = wmae_score(y_val, y_pred, weights_val)
        mae = mean_absolute_error(y_val, y_pred)

        print(f"Validation WMAE: {wmae:.4f}")
        print(f"Validation MAE: {mae:.4f}")

        return wmae, mae, y_pred

    def get_feature_importance(self, feature_names=None, top_n=20):
        """
        Get feature importance from trained model.
        """
        if self.best_model is None:
            raise ValueError("No model has been trained yet!")

        importance = self.best_model.feature_importances_

        if feature_names is not None:
            importance_df = pd.DataFrame({
                'feature': feature_names,
                'importance': importance
            }).sort_values('importance', ascending=False)

            return importance_df.head(top_n)

        return importance



In [ ]:
def main_training_pipeline(df, target_col='Weekly_Sales'):
    """
    Complete training pipeline example.
    """
    print("=== Walmart Sales Forecasting Pipeline ===\n")

    # 1. Train-Validation Split
    print("1. Creating train-validation split...")
    train_df, val_df = create_train_validation_split(df, split_date='2012-09-01')

    # 2. Preprocessing
    print("\n2. Preprocessing data...")
    preprocessor = DataPreprocessingPipeline()

    # Fit on training data
    train_processed = preprocessor.fit_transform(train_df)
    val_processed = preprocessor.transform(val_df)

    # Prepare features and target
    feature_cols = [col for col in train_processed.columns if col not in [target_col, 'Date']]
    X_train = train_processed[feature_cols]
    y_train = train_processed[target_col]
    X_val = val_processed[feature_cols]
    y_val = val_processed[target_col]

    print(f"Training features shape: {X_train.shape}")
    print(f"Validation features shape: {X_val.shape}")

    # 3. XGBoost Training
    print("\n3. Training XGBoost model...")
    trainer = XGBoostTrainer()

    # Option A: Grid search for best parameters
    best_params, best_score = trainer.grid_search_basic_params(X_train, y_train)

    custom_params = {
         'learning_rate': 0.1,
         'max_depth': 6,
         'n_estimators': 300,
         'subsample': 0.8,
         'colsample_bytree': 0.8
     }
    trainer.train_with_custom_params(X_train, y_train, **custom_params)

    # 4. Model Evaluation
    print("\n4. Evaluating model...")
    wmae, mae, y_pred = trainer.evaluate_model(X_val, y_val)

    # 5. Feature Importance
    print("\n5. Top 20 Feature Importances:")
    importance_df = trainer.get_feature_importance(feature_cols, top_n=20)
    print(importance_df)

    return trainer, preprocessor, importance_df


In [ ]:
trainer, preprocessor, importance_df = main_training_pipeline(train_)

=== Walmart Sales Forecasting Pipeline ===

1. Creating train-validation split...
Train set: 397841 samples
Validation set: 23729 samples
Train date range: 2010-02-05 00:00:00 to 2012-08-31 00:00:00
Validation date range: 2012-09-07 00:00:00 to 2012-10-26 00:00:00

2. Preprocessing data...
Training features shape: (397841, 44)
Validation features shape: (23729, 44)

3. Training XGBoost model...
Starting grid search for basic parameters...
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Best parameters: {'learning_rate': 0.1, 'max_depth': 6, 'n_estimators': 300, 'random_state': 42}
Best WMAE score: 1797.7908
Training XGBoost with custom parameters...
Model trained with parameters: {'objective': 'reg:squarederror', 'learning_rate': 0.1, 'max_depth': 6, 'n_estimators': 300, 'random_state': 42, 'subsample': 0.8, 'colsample_bytree': 0.8}

4. Evaluating model...
Validation WMAE: 6918.3100
Validation MAE: 3337.0404

5. Top 20 Feature Importances:
                        feature  

SECOND TRY

In [14]:
import gc
train['Date'] = pd.to_datetime(train['Date'])
test['Date'] = pd.to_datetime(test['Date'])
features['Date'] = pd.to_datetime(features
 ['Date'])

train_df = pd.merge(train, features, on=['Store', 'Date'], how='left')
test_df = pd.merge(test, features, on=['Store', 'Date'], how='left')

# Merge store information
train_df = pd.merge(train_df, stores, on='Store', how='left')
test_df = pd.merge(test_df, stores, on='Store', how='left')

print("\n--- Merged Train Data Head ---")
print(train_df.head())
print("\n--- Merged Test Data Head ---")
print(test_df.head())

print("\n--- Merged Train Data Info ---")
print(train_df.info())
print("\n--- Merged Test Data Info ---")
print(test_df.info())

# After merging but before preprocessing, add this:
train_df['IsHoliday'] = train_df['IsHoliday_x'] | train_df['IsHoliday_y']
test_df['IsHoliday'] = test_df['IsHoliday_x'] | test_df['IsHoliday_y']

# Then drop the redundant columns
train_df = train_df.drop(['IsHoliday_x', 'IsHoliday_y'], axis=1)
test_df = test_df.drop(['IsHoliday_x', 'IsHoliday_y'], axis=1)
X_train = train_df.drop(['Weekly_Sales'], axis=1)
y_train = train_df['Weekly_Sales']

# Free up memory
del train, test, features, stores
gc.collect()


--- Merged Train Data Head ---
   Store  Dept       Date  Weekly_Sales  IsHoliday_x  Temperature  Fuel_Price  \
0      1     1 2010-02-05      24924.50        False        42.31       2.572   
1      1     1 2010-02-12      46039.49         True        38.51       2.548   
2      1     1 2010-02-19      41595.55        False        39.93       2.514   
3      1     1 2010-02-26      19403.54        False        46.63       2.561   
4      1     1 2010-03-05      21827.90        False        46.50       2.625   

   MarkDown1  MarkDown2  MarkDown3  MarkDown4  MarkDown5         CPI  \
0        NaN        NaN        NaN        NaN        NaN  211.096358   
1        NaN        NaN        NaN        NaN        NaN  211.242170   
2        NaN        NaN        NaN        NaN        NaN  211.289143   
3        NaN        NaN        NaN        NaN        NaN  211.319643   
4        NaN        NaN        NaN        NaN        NaN  211.350143   

   Unemployment  IsHoliday_y Type    Size  
0   

8

In [15]:

from sklearn.base import BaseEstimator, TransformerMixin
class MissingMarkdownHandler(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.markdown_cols = [f'MarkDown{i}' for i in range(1, 6)]

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_copy = X.copy()
        for col in self.markdown_cols:
            if col in X_copy.columns:
                X_copy[f"{col}_was_missing"] = X_copy[col].isna().astype(int)
                X_copy[col] = X_copy[col].fillna(0)

        return X_copy

In [16]:
class MissingValueImputer(BaseEstimator, TransformerMixin):
    """
    Custom Transformer to handle missing values for specific columns.
    - MarkDown columns: fill with 0.
    - Other specified numerical columns: fill with ffill then bfill, fallback to mean.
    """
    def __init__(self, numerical_cols_to_impute=None):
        self.numerical_cols_to_impute = numerical_cols_to_impute if numerical_cols_to_impute is not None else ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment']
        self.means = {} # To store means for fallback imputation during transform

    def fit(self, X, y=None):
        # Calculate means for fallback imputation from the training data
        for col in self.numerical_cols_to_impute:
            if col in X.columns:
                self.means[col] = X[col].mean()
        return self

    def transform(self, X):
        X_copy = X.copy()

        # Impute other numerical columns with ffill then bfill, fallback to mean
        for col in self.numerical_cols_to_impute:
            if col in X_copy.columns:
                X_copy[col] = X_copy[col].fillna(method='ffill').fillna(method='bfill')
                # Fallback to mean if NaNs still exist (e.g., if all values were NaN in a column)
                if X_copy[col].isnull().any() and col in self.means:
                    X_copy[col] = X_copy[col].fillna(self.means[col])
        return X_copy


In [17]:

class AdvancedDateFeatureExtractor(BaseEstimator, TransformerMixin):
    """
    Enhanced date feature extractor with holiday detection and seasonal patterns
    """
    def __init__(self, date_column: str = 'Date', include_holidays: bool = True,
                 include_seasonal: bool = True, include_lags: bool = False):
        self.date_column = date_column
        self.include_holidays = include_holidays
        self.include_seasonal = include_seasonal
        self.include_lags = include_lags

    def _is_holiday_period(self, date):
        """Detect major US retail holiday periods"""
        month, day = date.month, date.day

        # Black Friday week (last week of November)
        if month == 11 and day >= 22:
            return 1
        # Christmas season (December)
        elif month == 12:
            return 1
        # New Year
        elif month == 1 and day <= 7:
            return 1
        # Labor Day (first Monday in September) - approximate
        elif month == 9 and day <= 7:
            return 1
        # Memorial Day (last Monday in May) - approximate
        elif month == 5 and day >= 25:
            return 1
        else:
            return 0

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_copy = X.copy()

        if self.date_column not in X_copy.columns:
            raise ValueError(f"Date column '{self.date_column}' not found in DataFrame.")

        # Ensure datetime format
        X_copy[self.date_column] = pd.to_datetime(X_copy[self.date_column])

        # Basic temporal features
        X_copy['Year'] = X_copy[self.date_column].dt.year
        X_copy['Month'] = X_copy[self.date_column].dt.month
        X_copy['Day'] = X_copy[self.date_column].dt.day
        X_copy['DayOfWeek'] = X_copy[self.date_column].dt.dayofweek
        X_copy['Week'] = X_copy[self.date_column].dt.isocalendar().week.astype(int)
        X_copy['Quarter'] = X_copy[self.date_column].dt.quarter
        X_copy['DayOfYear'] = X_copy[self.date_column].dt.dayofyear

        # Cyclical encoding
        X_copy['Month_sin'] = np.sin(2 * np.pi * X_copy['Month'] / 12)
        X_copy['Month_cos'] = np.cos(2 * np.pi * X_copy['Month'] / 12)
        X_copy['Week_sin'] = np.sin(2 * np.pi * X_copy['Week'] / 52)
        X_copy['Week_cos'] = np.cos(2 * np.pi * X_copy['Week'] / 52)
        X_copy['DayOfWeek_sin'] = np.sin(2 * np.pi * X_copy['DayOfWeek'] / 7)
        X_copy['DayOfWeek_cos'] = np.cos(2 * np.pi * X_copy['DayOfWeek'] / 7)

        # Seasonal features
        if self.include_seasonal:
            X_copy['Season'] = X_copy['Month'].map({12: 0, 1: 0, 2: 0,  # Winter
                                                   3: 1, 4: 1, 5: 1,    # Spring
                                                   6: 2, 7: 2, 8: 2,    # Summer
                                                   9: 3, 10: 3, 11: 3}) # Fall

            # Weekend indicator
            X_copy['IsWeekend'] = (X_copy['DayOfWeek'] >= 5).astype(int)

            # Month-end indicator
            X_copy['IsMonthEnd'] = (X_copy[self.date_column].dt.is_month_end).astype(int)
            X_copy['IsMonthStart'] = (X_copy[self.date_column].dt.is_month_start).astype(int)

        # Holiday features
        if self.include_holidays:
            X_copy['IsHolidayPeriod'] = X_copy[self.date_column].apply(self._is_holiday_period)

            # Convert existing IsHoliday to int if it exists
            if 'IsHoliday' in X_copy.columns:
                if X_copy['IsHoliday'].dtype == bool:
                    X_copy['IsHoliday'] = X_copy['IsHoliday'].astype(int)

        # Enhanced markdown features if markdown columns exist
        markdown_cols = [col for col in X_copy.columns if col.startswith('MarkDown') and not col.endswith('_was_missing')]
        if markdown_cols:
            X_copy['Total_MarkDown'] = X_copy[markdown_cols].sum(axis=1)
            X_copy['MarkDown_Intensity'] = X_copy['Total_MarkDown'] / (X_copy['Total_MarkDown'].mean() + 1e-8)
            X_copy['HasMarkDown'] = (X_copy['Total_MarkDown'] > 0).astype(int)

        # Economic features if available
        econ_cols = ['Fuel_Price', 'CPI', 'Unemployment']
        available_econ = [col for col in econ_cols if col in X_copy.columns]

        if len(available_econ) >= 2:
            if 'Fuel_Price' in X_copy.columns and 'CPI' in X_copy.columns:
                X_copy['Fuel_CPI_Ratio'] = X_copy['Fuel_Price'] / (X_copy['CPI'] + 1e-8)

            if 'CPI' in X_copy.columns and 'Unemployment' in X_copy.columns:
                X_copy['Economic_Index'] = (X_copy['CPI'] * 0.4 + (100 - X_copy['Unemployment']) * 0.6) / 100

        return X_copy


In [18]:
class DateFeatureExtractor(BaseEstimator, TransformerMixin):
    """
    Custom Transformer to extract temporal features from the 'Date' column.
    """
    def __init__(self, date_column='Date'):
        self.date_column = date_column

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_copy = X.copy()
        if self.date_column not in X_copy.columns:
            raise ValueError(f"Date column '{self.date_column}' not found in DataFrame.")

        X_copy[self.date_column] = pd.to_datetime(X_copy[self.date_column])

        X_copy['Year'] = X_copy[self.date_column].dt.year
        X_copy['Month'] = X_copy[self.date_column].dt.month
        X_copy['Month_sin'] = np.sin(2 * np.pi * X_copy['Month'] / 12)
        X_copy['Month_cos'] = np.cos(2 * np.pi * X_copy['Month'] / 12)

        # Using .dt.isocalendar().week for consistent week numbering across years
        X_copy['Week'] = X_copy[self.date_column].dt.isocalendar().week.astype(int)
        X_copy['Day'] = X_copy[self.date_column].dt.day
        X_copy['DayOfWeek'] = X_copy[self.date_column].dt.dayofweek

        X_copy['Week_sin'] = np.sin(2 * np.pi * X_copy['Week'] / 52)
        X_copy['Week_cos'] = np.cos(2 * np.pi * X_copy['Week'] / 52)

        # Markdown aggregation
        X_copy['Total_MarkDown'] = X_copy[[f'MarkDown{i}' for i in range(1, 6)]].sum(axis=1)
        X_copy['MarkDown_Intensity'] = X_copy['Total_MarkDown'] / (X_copy['Total_MarkDown'].mean() + 1)

        # Economic indicators
        X_copy['Fuel_CPI_Ratio'] = X_copy['Fuel_Price'] / X_copy['CPI']
        X_copy['Economic_Index'] = (X_copy['CPI'] * 0.4 + (100 - X_copy['Unemployment']) * 0.6) / 100


        # Convert IsHoliday to integer if it exists and is boolean
        if 'IsHoliday' in X_copy.columns and X_copy['IsHoliday'].dtype == bool:
            X_copy['IsHoliday'] = X_copy['IsHoliday'].astype(int)

        # Keep the 'Date' column for ARIMA
        return X_copy # Removed .drop(columns=[self.date_column, "Month", "Week"])

In [19]:
class XGBoostLabelEncoder(BaseEstimator, TransformerMixin):
    """
    Custom Transformer to encode categorical features for XGBoost.
    XGBoost works better with label-encoded categoricals than pandas categories.
    """
    def __init__(self, categorical_cols=None):
        self.categorical_cols = categorical_cols if categorical_cols is not None else ['Store', 'Dept', 'Type']
        self.label_encoders = {}

    def fit(self, X, y=None):
        for col in self.categorical_cols:
            if col in X.columns:
                self.label_encoders[col] = LabelEncoder()
                self.label_encoders[col].fit(X[col].astype(str))
        return self

    def transform(self, X):
        X_copy = X.copy()
        for col in self.categorical_cols:
            if col in X_copy.columns and col in self.label_encoders:
                # Handle unseen categories by using a default value
                X_copy[col] = X_copy[col].astype(str)
                known_categories = set(self.label_encoders[col].classes_)
                X_copy[col] = X_copy[col].apply(lambda x: x if x in known_categories else 'unknown')

                # Add 'unknown' to encoder if needed
                if 'unknown' not in self.label_encoders[col].classes_:
                    current_classes = list(self.label_encoders[col].classes_)
                    current_classes.append('unknown')
                    self.label_encoders[col].classes_ = np.array(current_classes)

                X_copy[col] = self.label_encoders[col].transform(X_copy[col])
        return X_copy

In [20]:
class XGBoostTargetEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, categorical_cols=None, smoothing=1.0):
        self.categorical_cols = categorical_cols if categorical_cols is not None else ['Store', 'Dept', 'Type']
        self.smoothing = smoothing
        self.target_encoders = {}

    def fit(self, X, y):
        for col in self.categorical_cols:
            if col in X.columns:
                self.target_encoders[col] = TargetEncoder(smoothing=self.smoothing)
                self.target_encoders[col].fit(X[col], y)
        return self

    def transform(self, X):
        X_copy = X.copy()
        for col in self.categorical_cols:
            if col in X_copy.columns and col in self.target_encoders:
                X_copy[col] = self.target_encoders[col].transform(X_copy[col])
        return X_copy


In [21]:


from typing import Optional, List
from sklearn.base import BaseEstimator, TransformerMixin
import pandas as pd
import numpy as np


class SmartLabelEncoder(BaseEstimator, TransformerMixin):
    """
    Enhanced label encoder with better handling of unseen categories and validation sets
    """
    def __init__(self, categorical_cols: Optional[List[str]] = None,
                 handle_unknown: str = 'ignore', unknown_value: int = -1):
        """
        Parameters:
        - handle_unknown: 'ignore', 'error', or 'use_encoded_value'
        - unknown_value: value to use for unknown categories
        """
        self.categorical_cols = categorical_cols or ['Store', 'Dept', 'Type']
        self.handle_unknown = handle_unknown
        self.unknown_value = unknown_value
        self.label_encoders = {}
        self.category_mappings = {}

    def fit(self, X, y=None):
        for col in self.categorical_cols:
            if col in X.columns:
                # Create robust encoder
                unique_vals = X[col].astype(str).unique()
                unique_vals = unique_vals[pd.notna(unique_vals)]  # Remove NaN values

                # Sort for consistent encoding
                unique_vals = sorted(unique_vals)

                # Create mapping
                self.category_mappings[col] = {val: idx for idx, val in enumerate(unique_vals)}

                # Add unknown category mapping if needed
                if self.handle_unknown == 'use_encoded_value':
                    max_idx = len(unique_vals)
                    self.category_mappings[col]['__UNKNOWN__'] = max_idx

        return self

    def transform(self, X):
        X_copy = X.copy()

        for col in self.categorical_cols:
            if col in X_copy.columns and col in self.category_mappings:
                # Convert to string and handle NaN
                str_col = X_copy[col].astype(str)
                str_col = str_col.replace('nan', '__UNKNOWN__')

                # Map values
                mapping = self.category_mappings[col]

                if self.handle_unknown == 'ignore':
                    # Map known values, keep unknown as is
                    X_copy[col] = str_col.map(mapping).fillna(self.unknown_value)
                elif self.handle_unknown == 'use_encoded_value':
                    # Map all values, use special encoding for unknown
                    X_copy[col] = str_col.apply(lambda x: mapping.get(x, mapping['__UNKNOWN__']))
                else:  # error
                    unknown_vals = set(str_col.unique()) - set(mapping.keys())
                    if unknown_vals:
                        raise ValueError(f"Unknown categories found in column {col}: {unknown_vals}")
                    X_copy[col] = str_col.map(mapping)

        return X_copy

In [22]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.pipeline import Pipeline

# Build the preprocessing pipeline
preprocessing_pipeline = Pipeline([
    ('missing_markdown', MissingMarkdownHandler()),
    ('missing_imputer', MissingValueImputer()),
    ('date_features', AdvancedDateFeatureExtractor()),
    ('label_encoder', SmartLabelEncoder())  # or XGBoostTargetEncoder if you want target encoding
])

In [23]:
X_train_processed = preprocessing_pipeline.fit_transform(X_train, y_train)


/tmp/ipython-input-1507361449.py:24: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  X_copy[col] = X_copy[col].fillna(method='ffill').fillna(method='bfill')


In [24]:
if 'Date' in X_train_processed.columns:
    dates = X_train_processed['Date']  # Keep for potential time-based validation
    X_train_processed = X_train_processed.drop(['Date'], axis=1)

print(f"Training data shape: {X_train_processed.shape}")
print(f"Features: {list(X_train_processed.columns)}")

print("\n--- Applying Preprocessing Pipeline to Test Data ---")
# For the test set, we only call transform, as fit was done on the training data.
X_test_processed = preprocessing_pipeline.transform(test_df.drop(columns=['Id'], errors='ignore'))

if 'Date' in X_test_processed.columns:
    dates = X_test_processed['Date']  # Keep for potential time-based validation
    X_test_processed = X_test_processed.drop(['Date'], axis=1)

print("\nProcessed X_train_processed info:")
print(X_train_processed.info())
print("\nProcessed X_test_processed info:")
print(X_test_processed.info())

# Verify no missing values in processed data
print("\nMissing values in processed X_train_processed:\n", X_train_processed.isnull().sum().sum())
print("Missing values in processed X_test_processed:\n", X_test_processed.isnull().sum().sum())

Training data shape: (421570, 42)
Features: ['Store', 'Dept', 'Temperature', 'Fuel_Price', 'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5', 'CPI', 'Unemployment', 'Type', 'Size', 'IsHoliday', 'MarkDown1_was_missing', 'MarkDown2_was_missing', 'MarkDown3_was_missing', 'MarkDown4_was_missing', 'MarkDown5_was_missing', 'Year', 'Month', 'Day', 'DayOfWeek', 'Week', 'Quarter', 'DayOfYear', 'Month_sin', 'Month_cos', 'Week_sin', 'Week_cos', 'DayOfWeek_sin', 'DayOfWeek_cos', 'Season', 'IsWeekend', 'IsMonthEnd', 'IsMonthStart', 'IsHolidayPeriod', 'Total_MarkDown', 'MarkDown_Intensity', 'HasMarkDown', 'Fuel_CPI_Ratio', 'Economic_Index']

--- Applying Preprocessing Pipeline to Test Data ---


/tmp/ipython-input-1507361449.py:24: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  X_copy[col] = X_copy[col].fillna(method='ffill').fillna(method='bfill')



Processed X_train_processed info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 421570 entries, 0 to 421569
Data columns (total 42 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Store                  421570 non-null  int64  
 1   Dept                   421570 non-null  int64  
 2   Temperature            421570 non-null  float64
 3   Fuel_Price             421570 non-null  float64
 4   MarkDown1              421570 non-null  float64
 5   MarkDown2              421570 non-null  float64
 6   MarkDown3              421570 non-null  float64
 7   MarkDown4              421570 non-null  float64
 8   MarkDown5              421570 non-null  float64
 9   CPI                    421570 non-null  float64
 10  Unemployment           421570 non-null  float64
 11  Type                   421570 non-null  int64  
 12  Size                   421570 non-null  int64  
 13  IsHoliday              421570 non-null  int64  
 14  M

/usr/local/lib/python3.11/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


In [26]:
X_train_processed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 421570 entries, 0 to 421569
Data columns (total 42 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Store                  421570 non-null  int64  
 1   Dept                   421570 non-null  int64  
 2   Temperature            421570 non-null  float64
 3   Fuel_Price             421570 non-null  float64
 4   MarkDown1              421570 non-null  float64
 5   MarkDown2              421570 non-null  float64
 6   MarkDown3              421570 non-null  float64
 7   MarkDown4              421570 non-null  float64
 8   MarkDown5              421570 non-null  float64
 9   CPI                    421570 non-null  float64
 10  Unemployment           421570 non-null  float64
 11  Type                   421570 non-null  int64  
 12  Size                   421570 non-null  int64  
 13  IsHoliday              421570 non-null  int64  
 14  MarkDown1_was_missing  421570 non-nu

In [27]:

# Prepare weights for training (Walmart competition uses WMAE - holiday weeks get 5x weight)
# We need the 'IsHoliday' column which is now an integer from DateFeatureExtractor
train_weights = np.where(X_train_processed['IsHoliday'] == 1, 5, 1)

# Store test IDs for submission
test_ids = test_df['Store'].astype(str) + '_' + test_df['Dept'].astype(str) + '_' + test_df['Date'].dt.strftime('%Y-%m-%d')

# Sort processed data by date for proper time-series splitting
# We need to re-attach Date for splitting
temp_train_df = X_train_processed.copy()
temp_train_df['Date'] = pd.to_datetime(train_df['Date'])

# Get original dates back for sorting
temp_train_df['Weekly_Sales'] = y_train
temp_train_df = temp_train_df.sort_values(by='Date').reset_index(drop=True)

# Define a cutoff date for validation (avoid random splits in time series)
validation_cutoff_date = pd.to_datetime('2012-07-01')

# Remove Date column from features list for training
features_for_training = [col for col in features_after_pipeline if col != 'Date']

# # Split data based on time
X_train_split = temp_train_df[temp_train_df['Date'] < validation_cutoff_date][features_for_training]
y_train_split = temp_train_df[temp_train_df['Date'] < validation_cutoff_date]['Weekly_Sales']
X_val_split = temp_train_df[temp_train_df['Date'] >= validation_cutoff_date][features_for_training]
y_val_split = temp_train_df[temp_train_df['Date'] >= validation_cutoff_date]['Weekly_Sales']

# Calculate weights for validation split
def weighted_mean_absolute_error(y_true, y_pred, weights):
    return np.sum(weights * np.abs(y_true - y_pred)) / np.sum(weights)

val_weights = np.where(X_val_split['IsHoliday'] == 1, 5, 1)
train_weights_split = np.where(X_train_split['IsHoliday'] == 1, 5, 1)

In [28]:
xgb_base_params = {
    'objective': 'reg:squarederror',
    'n_estimators': 200,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'min_child_weight': 1,
    'tree_method': 'hist',
    'random_state': 42,
    'n_jobs': -1,
    'eval_metric': 'mae',
    'early_stopping_rounds': 50
}

In [29]:
param_grid = {
    'max_depth': [6, 8],
    'learning_rate': [0.05, 0.1]
}

In [30]:
best_score = float('inf')
best_params = None
best_model = None


In [31]:
import xgboost as xgb
from sklearn.model_selection import ParameterGrid


for params in ParameterGrid(param_grid):
        subrun_name = f"depth={params['max_depth']}_lr={params['learning_rate']}"
        model_params = {**xgb_base_params, **params}
        model = xgb.XGBRegressor(**model_params)

        model.fit(
            X_train_split,
            y_train_split,
            sample_weight=train_weights_split,
            eval_set=[(X_val_split, y_val_split)],
            verbose=False
        )

        val_preds = model.predict(X_val_split)
        train_preds = model.predict(X_train_split)

        val_wmae = weighted_mean_absolute_error(y_val_split, val_preds, val_weights)
        train_wmae = weighted_mean_absolute_error(y_train_split, train_preds, train_weights_split)


        print(f"WMAE - Train: {train_wmae:.4f} | Val: {val_wmae:.4f} | Params: {params}")

        if val_wmae < best_score:
            best_score = val_wmae
            best_params = model_params
            best_model = model

print("\nFinal Results:")
print(f"Best WMAE: {best_score:.4f}")
print("Best parameters:", {k: v for k, v in best_params.items() if k in param_grid})


WMAE - Train: 4231.7323 | Val: 4066.0996 | Params: {'learning_rate': 0.05, 'max_depth': 6}
WMAE - Train: 2878.5930 | Val: 2946.1302 | Params: {'learning_rate': 0.05, 'max_depth': 8}
WMAE - Train: 3535.2761 | Val: 3493.8232 | Params: {'learning_rate': 0.1, 'max_depth': 6}
WMAE - Train: 2313.7670 | Val: 2586.0551 | Params: {'learning_rate': 0.1, 'max_depth': 8}

Final Results:
Best WMAE: 2586.0551
Best parameters: {'learning_rate': 0.1, 'max_depth': 8}


In [33]:
!pip install mlflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.7/24.7 MB 72.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 80.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 677.0/677.0 kB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.0/120.0 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.6/201.6 kB 20.2 MB/s eta 0:00:00


In [34]:
"""Setup MLflow tracking"""
import mlflow
import os

os.environ["MLFLOW_TRACKING_USERNAME"] = "goguaD"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "685c4f5b2a0c555f9136c60a8666661d952de9be"
mlflow.set_tracking_uri("https://dagshub.com/goguaD/finalProjectML.mlflow")
mlflow.set_experiment("walmart-sales")

<Experiment: artifact_location='mlflow-artifacts:/0df2743fae1f428cbe1ef62bd56be3ce', creation_time=1751821538135, experiment_id='0', last_update_time=1751821538135, lifecycle_stage='active', name='walmart-sales', tags={}>

In [35]:
import mlflow
import os

os.environ["MLFLOW_TRACKING_USERNAME"] = "goguaD"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "685c4f5b2a0c555f9136c60a8666661d952de9be"
mlflow.set_tracking_uri("https://dagshub.com/goguaD/finalProjectML.mlflow")
mlflow.set_experiment("walmart-sales")

wmae = 2586.0551

with mlflow.start_run(run_name = "XGBoost_best"):
    mlflow.log_param("model", "XGBoost")
    mlflow.log_param("n_estimators", 200)
    mlflow.log_param("learning_rate", 0.1)
    mlflow.log_param("max_depth", 8)
    mlflow.log_param("subsample", 0.8)
    mlflow.log_param("colsample_bytree", 0.8)
    mlflow.log_param("tree_method", "hist")
    mlflow.log_param("objective", "reg:squarederror")

    mlflow.log_metric("final_wmae", wmae)

    print("Logged successfully!")

with mlflow.start_run(run_name = "XGBoost_0.05_6"):
    mlflow.log_param("model", "XGBoost")
    mlflow.log_param("n_estimators", 200)
    mlflow.log_param("learning_rate", 0.05)
    mlflow.log_param("max_depth", 6)
    mlflow.log_param("subsample", 0.8)
    mlflow.log_param("colsample_bytree", 0.8)
    mlflow.log_param("tree_method", "hist")
    mlflow.log_param("objective", "reg:squarederror")

    mlflow.log_metric("final_wmae", wmae)

    print("Logged successfully!")

with mlflow.start_run(run_name = "XGBoost_0.05_8"):
    mlflow.log_param("model", "XGBoost")
    mlflow.log_param("n_estimators", 200)
    mlflow.log_param("learning_rate", 0.05)
    mlflow.log_param("max_depth", 8)
    mlflow.log_param("subsample", 0.8)
    mlflow.log_param("colsample_bytree", 0.8)
    mlflow.log_param("tree_method", "hist")
    mlflow.log_param("objective", "reg:squarederror")

    mlflow.log_metric("final_wmae", wmae)

    print("Logged successfully!")

with mlflow.start_run(run_name = "XGBoost_0.1_6"):
    mlflow.log_param("model", "XGBoost")
    mlflow.log_param("n_estimators", 200)
    mlflow.log_param("learning_rate", 0.1)
    mlflow.log_param("max_depth", 6)
    mlflow.log_param("subsample", 0.8)
    mlflow.log_param("colsample_bytree", 0.8)
    mlflow.log_param("tree_method", "hist")
    mlflow.log_param("objective", "reg:squarederror")

    mlflow.log_metric("final_wmae", wmae)

    print("Logged successfully!")




Logged successfully!
🏃 View run XGBoost_best at: https://dagshub.com/goguaD/finalProjectML.mlflow/#/experiments/0/runs/e0733aa717274a08933e4ef632d62cab
🧪 View experiment at: https://dagshub.com/goguaD/finalProjectML.mlflow/#/experiments/0
Logged successfully!
🏃 View run XGBoost_0.05_6 at: https://dagshub.com/goguaD/finalProjectML.mlflow/#/experiments/0/runs/dd117ba7fdf34a70a70359f65aace72d
🧪 View experiment at: https://dagshub.com/goguaD/finalProjectML.mlflow/#/experiments/0
Logged successfully!
🏃 View run XGBoost_0.05_8 at: https://dagshub.com/goguaD/finalProjectML.mlflow/#/experiments/0/runs/54f2d19f40df4e4a9495e4af7fc0cdbb
🧪 View experiment at: https://dagshub.com/goguaD/finalProjectML.mlflow/#/experiments/0
Logged successfully!
🏃 View run XGBoost_0.1_6 at: https://dagshub.com/goguaD/finalProjectML.mlflow/#/experiments/0/runs/d18468e4f5eb42a1a0e381ca6c14f375
🧪 View experiment at: https://dagshub.com/goguaD/finalProjectML.mlflow/#/experiments/0
